| **Operation**   | **New/Shared memory** | **Still in computation graph** |
| --------------- | --------------------- | ------------------------------ |
| tor.clone()  | New                   | Yes                            |
| tensor.detach() | Shared                | No                             |
| tensor.clone().detach() | New                | No                            |

### torch.clone

In [26]:
import torch

a = torch.tensor(1.0, requires_grad=True)
a_y = a ** 2

acl = torch.clone(a)  # Gradients propagating to the cloned tensor will propagate to the original tensor.
acl.retain_grad()
print(acl)
acl_y = acl * 3

tensor(1., grad_fn=<CloneBackward>)


In [27]:
a_y.backward()
print(a.grad, end='\n\n')

acl_y.backward(retain_graph=True)
print(acl.grad)  # acl为非叶tensor,通过acl_y.retain_grad()可输出acl.grad
print(a.grad, end='\n\n')  # acl.grad会传递回给a.grad,因此a.grad=a.grad(2)+acl.grad(3)=5

acl_y.backward()
print(acl.grad)  # acl.grad由3累加到6
print(a.grad)  # 此时a.grad=a.grad(2)+acl.grad(6)=8

tensor(2.)

tensor(3.)
tensor(5.)

tensor(6.)
tensor(8.)


In [28]:
acl.fill_(0)

print(acl)
print(a)  # 不共享数据的物理内存

tensor(0., grad_fn=<FillBackward0>)
tensor(1., requires_grad=True)


### torch.detach

In [29]:
b = torch.tensor([1., 2., 3., 4., 5., 6.], requires_grad=True)

bde = torch.detach(b)
print(bde)  # requires_grad=False,脱离计算图

tensor([1., 2., 3., 4., 5., 6.])


In [30]:
# detach函数与data属性的比较参考tensor相关属性.ipynb;backward注意1.ipynb
bde.add_(10)  # 不能改变bde的地址

print(bde)
print(b)  # 共享数据的物理内存

tensor([11., 12., 13., 14., 15., 16.])
tensor([11., 12., 13., 14., 15., 16.], requires_grad=True)


In [31]:
print(id(bde))
bde = bde + 100
print(id(bde))  # bde地址改变
print(b)  # b不变

2319213526272
2319211893952
tensor([11., 12., 13., 14., 15., 16.], requires_grad=True)
